In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from itertools import product

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

import pandas as pd
import seaborn as sns

import sbibm
from sbibm.utils.io import get_tensor_from_csv

import sbi.analysis as analysis

from notebook import nre_gamma_default, prepare_df_for_plot, get_metrics, wide_to_long

In [3]:
df = get_metrics(
    [
        "fix-sbibm.csv",
    ],
    "sbibm",
)
# df_extra = get_metrics(
#     [
#         "metrics-sbibm-extra.csv",
#     ],
#     "sbibm",
# )
# df_extra_extra = get_metrics(
#     [
#         "metrics-sbibm-extra-extra.csv",
#     ],
#     "sbibm",
# )

# df = pd.concat([df, df_extra, df_extra_extra], ignore_index=True)
df["gamma"] = df["gamma"].fillna(nre_gamma_default)

print(len(df))
print(df.columns)
print(df["algorithm"].unique())
print(df["task"].unique())
print(df["num_contrastive_parameters"].unique())
df.head()

173
Index(['task', 'seed', 'algorithm', 'K', 'num_atoms',
       'num_contrastive_parameters', 'gamma', 'extra_theta_factor', 'reuse',
       'hidden_features', 'num_blocks', 'AVG_LOG_RATIO', 'RT', 'C2ST_Z-01',
       'MMD-01', 'MEDDIST-01', 'NLTP-01', 'C2ST_Z-02', 'MMD-02', 'MEDDIST-02',
       'NLTP-02', 'C2ST_Z-03', 'MMD-03', 'MEDDIST-03', 'NLTP-03', 'C2ST_Z-04',
       'MMD-04', 'MEDDIST-04', 'NLTP-04', 'C2ST_Z-05', 'MMD-05', 'MEDDIST-05',
       'NLTP-05', 'C2ST_Z-06', 'MMD-06', 'MEDDIST-06', 'NLTP-06', 'C2ST_Z-07',
       'MMD-07', 'MEDDIST-07', 'NLTP-07', 'C2ST_Z-08', 'MMD-08', 'MEDDIST-08',
       'NLTP-08', 'C2ST_Z-09', 'MMD-09', 'MEDDIST-09', 'NLTP-09', 'C2ST_Z-10',
       'MMD-10', 'MEDDIST-10', 'NLTP-10', 'NLTP', 'num_simulations_simulator',
       'path', 'folder', 'expr'],
      dtype='object')
['CNRE']
['lotka_volterra' 'gaussian_linear' 'bernoulli_glm_raw' 'slcp_distractors'
 'two_moons' 'gaussian_mixture' 'bernoulli_glm' 'slcp'
 'gaussian_linear_uniform' 'sir']
[100]


,task,seed,algorithm,K,num_atoms,num_contrastive_parameters,gamma,extra_theta_factor,reuse,hidden_features,...,NLTP-09,C2ST_Z-10,MMD-10,MEDDIST-10,NLTP-10,NLTP,num_simulations_simulator,path,folder,expr
0,lotka_volterra,960951598,CNRE,99,99,100,1.0,0,False,128,...,NaN,0.95565,0.350041,29.014172,NaN,NaN,100000.0,/home/ben/sci/fix-cnre/infinite/sbibm/2024-07-...,5,sbibm
1,lotka_volterra,2738266988,CNRE,99,99,100,1.0,0,False,128,...,NaN,0.99770,0.503656,119.054910,NaN,NaN,1000.0,/home/ben/sci/fix-cnre/infinite/sbibm/2024-07-...,9,sbibm
2,lotka_volterra,2227534949,CNRE,99,99,100,1.0,0,False,128,...,NaN,0.99585,0.469641,52.861230,NaN,NaN,10000.0,/home/ben/sci/fix-cnre/infinite/sbibm/2024-07-...,10,sbibm
3,lotka_volterra,2256148287,CNRE,99,99,100,1.0,0,False,128,...,NaN,0.99205,0.444111,36.822464,NaN,NaN,10000.0,/home/ben/sci/fix-cnre/infinite/sbibm/2024-07-...,1,sbibm
4,lotka_volterra,3549010460,CNRE,99,99,100,1.0,0,False,128,...,NaN,0.98660,0.449134,39.344720,NaN,NaN,10000.0,/home/ben/sci/fix-cnre/infinite/sbibm/2024-07-...,4,sbibm


## count the experiments.

In [4]:
grp = df.groupby(["task", "num_simulations_simulator"])
grp.count()

# ddd = grp.head(4)
# ddd.groupby(["task", "num_simulations_simulator"]).count()

seed  algorithm   K  \
task                    num_simulations_simulator                        
bernoulli_glm           1000.0                        5          5   5   
                        10000.0                       5          5   5   
                        100000.0                      5          5   5   
bernoulli_glm_raw       1000.0                        5          5   5   
                        10000.0                       5          5   5   
                        100000.0                      5          5   5   
gaussian_linear         1000.0                        5          5   5   
                        10000.0                       5          5   5   
                        100000.0                      5          5   5   
gaussian_linear_uniform 1000.0                        5          5   5   
                        10000.0                       5          5   5   
                        100000.0                      5          5   5   
gaussian_mixture        1000.0                        8          8   8   
                        10000.0                      10         10  10   
                        100000.0                     10         10  10   
lotka_volterra          1000.0                        5          5   5   
                        10000.0                       5          5   5   
                        100000.0                      5          5   5   
sir                     1000.0                        5          5   5   
                        10000.0                       5          5   5   
                        100000.0                      5          5   5   
slcp                    1000.0                        5          5   5   
                        10000.0                       5          5   5   
                        100000.0                      5          5   5   
slcp_distractors        1000.0                       10         10  10   
                        10000.0                      10         10  10   
                        100000.0                      5          5   5   
two_moons               1000.0                        5          5   5   
                        10000.0                       5          5   5   
                        100000.0                      5          5   5   

                                                   num_atoms  \
task                    num_simulations_simulator              
bernoulli_glm           1000.0                             5   
                        10000.0                            5   
                        100000.0                           5   
bernoulli_glm_raw       1000.0                             5   
                        10000.0                            5   
                        100000.0                           5   
gaussian_linear         1000.0                             5   
                        10000.0                            5   
                        100000.0                           5   
gaussian_linear_uniform 1000.0                             5   
                        10000.0                            5   
                        100000.0                           5   
gaussian_mixture        1000.0                             8   
                        10000.0                           10   
                        100000.0                          10   
lotka_volterra          1000.0                             5   
                        10000.0                            5   
                        100000.0                           5   
sir                     1000.0                             5   
                        10000.0                            5   
                        100000.0                           5   
slcp                    1000.0                             5   
                        10000.0                            5   
                        100000.0                           5   
slcp_distractors  

In [5]:
# dfs = []
# for i in range(5):
#     dfs.append(grp.nth(i).reset_index())
# df_5 = pd.concat(dfs).reset_index()
# print(df_5.shape)

# grp5 = df_5.groupby(["task", "num_simulations_simulator"])
# c2st_names = [c for c in df.columns if "C2ST" in c]
# cnt = grp5.count()
# for cn in c2st_names:
#     assert (cnt[cn] == 5).all(), f"{cn} did not have 5 values"

In [6]:
# df = df_5
# df = df.drop("level_0", axis=1)
# df = df.drop("index", axis=1)

## do some averaging

In [7]:
df_wide = pd.wide_to_long(
    df,
    ["C2ST_Z"],
    i=[
        "task",
        "algorithm",
        "num_contrastive_parameters",
        "gamma",
        "num_blocks",
        "hidden_features",
        "seed",
    ],
    j="num_observation",
    sep="-",
)

# groupby drops NaNs
# grp = df_wide.drop(["expr", "path"], axis=1).groupby(
grp = df_wide.groupby(
    [
        "task",
        "algorithm",
        "num_simulations_simulator"
    ]
)

#### this next section truncates to 5 experiments so all experiments use the same number of initial seeds!

In [8]:
# TAKE THE FIRST 5 ONLY!!!
temp = grp.head(5)
grp = temp.groupby(
    [
        "task",
        "algorithm",
        "num_simulations_simulator"
    ]
)
c2st_by_task_num_sim = grp.mean(numeric_only=True)["C2ST_Z"]

In [9]:
out = c2st_by_task_num_sim.groupby(["task", "num_simulations_simulator"]).mean()
out = out.to_frame().reset_index()
out

,task,num_simulations_simulator,C2ST_Z
0,bernoulli_glm,1000.0,0.80206
1,bernoulli_glm,10000.0,0.69373
2,bernoulli_glm,100000.0,0.60089
3,bernoulli_glm_raw,1000.0,0.92816
4,bernoulli_glm_raw,10000.0,0.77499
5,bernoulli_glm_raw,100000.0,0.60875
6,gaussian_linear,1000.0,0.65535
7,gaussian_linear,10000.0,0.55646
8,gaussian_linear,100000.0,0.51850
9,gaussian_linear_uniform,1000.0,0.74566


In [10]:
from_sims_to_sym = {1000.0: "10³", 10000.0: "10⁴", 100000.0: "10⁵"}
out["num_simulations"] = out["num_simulations_simulator"].map(from_sims_to_sym)
out["C2ST"] = out["C2ST_Z"]
out["algorithm"] = "NRE-C"

## load sbibm results

In [11]:
mp = pd.read_csv("../remote/results/benchmarking_sbi/results/main_paper.csv")
results = mp.groupby(["task", "algorithm", "num_simulations"]).mean(numeric_only=True).reset_index()

In [12]:
cat = pd.concat(
    [results[["algorithm", "task", "num_simulations", "C2ST"]], out[["algorithm", "task", "num_simulations", "C2ST"]]], 
    ignore_index=True
)

cat["algorithm"] = cat["algorithm"].map(lambda x: "\\" + "".join(x.split("-")))
cat["algorithm"] = cat["algorithm"].map(lambda x: "\\NREC (ours)" if x == "\\NREC" else x)
cat["algorithm"] = cat["algorithm"].map(lambda x: "\\NRE (\\NREB)" if x == "\\NRE" else x)
cat["algorithm"] = cat["algorithm"].map(lambda x: "\\SNRE (\\SNREB)" if x == "\\SNRE" else x)

cat["Simulation budget"] = cat["num_simulations"]
cat = cat.drop("num_simulations", axis=1)
from_sym_to_sims = {"10³": r"$10^3$", "10⁴": r"$10^4$", "10⁵": r"$10^5$"}
cat["Simulation budget"] = cat["Simulation budget"].map(from_sym_to_sims)

cat["Algorithm"] = cat["algorithm"]
cat = cat.drop("algorithm", axis=1)

cat["Task"] = cat["task"].map(lambda x: sbibm.tasks.get_task(x).name_display)
cat = cat.drop("task", axis=1)

## short summary

In [13]:
table = cat.drop("Task", axis=1).groupby(["Algorithm", "Simulation budget"]).mean().unstack()

order = [
    '\\NREC (ours)',
    '\\REJABC', 
    '\\NLE', 
    '\\NPE', 
    '\\NRE (\\NREB)',  
    '\\SMCABC', 
    '\\SNLE', 
    '\\SNPE',
    '\\SNRE (\\SNREB)',
]
table = table.loc[order]

print(
    table.style.format(precision=3).to_latex(
        position_float="centering",
        hrules=True,
        caption="Simulation-based inference benchmark results.",
        label="table:sbibm"
    )
)

\begin{table}
\centering
\caption{Simulation-based inference benchmark results.}
\label{table:sbibm}
\begin{tabular}{lrrr}
\toprule
 & \multicolumn{3}{r}{C2ST} \\
Simulation budget & $10^3$ & $10^4$ & $10^5$ \\
Algorithm &  &  &  \\
\midrule
\NREC (ours) & 0.828 & 0.746 & 0.705 \\
\REJABC & 0.965 & 0.920 & 0.871 \\
\NLE & 0.826 & 0.753 & 0.723 \\
\NPE & 0.838 & 0.736 & 0.654 \\
\NRE (\NREB) & 0.867 & 0.811 & 0.762 \\
\SMCABC & 0.948 & 0.873 & 0.816 \\
\SNLE & 0.783 & 0.704 & 0.655 \\
\SNPE & 0.796 & 0.677 & 0.615 \\
\SNRE (\SNREB) & 0.788 & 0.703 & 0.610 \\
\bottomrule
\end{tabular}
\end{table}



## long summary

In [14]:
table = cat.groupby(["Task", "Algorithm", "Simulation budget"]).first().unstack()

order = [
    '\\NREC (ours)',
    '\\REJABC', 
    '\\NLE', 
    '\\NPE', 
    '\\NRE (\\NREB)',  
    '\\SMCABC', 
    '\\SNLE', 
    '\\SNPE',
    '\\SNRE (\\SNREB)',
]
multi_order = list(product(table.index.get_level_values(0).unique().tolist(), order))
table = table.loc[multi_order]

print(
    table.style.format(precision=3).to_latex(
        position_float="centering",
        hrules=True,
        caption="Simulation-based inference benchmark results.",
        label="table:sbibm-details",
        clines="skip-last;data",
    )
)

\begin{table}
\centering
\caption{Simulation-based inference benchmark results.}
\label{table:sbibm-details}
\begin{tabular}{llrrr}
\toprule
 &  & \multicolumn{3}{r}{C2ST} \\
 & Simulation budget & $10^3$ & $10^4$ & $10^5$ \\
Task & Algorithm &  &  &  \\
\midrule
\multirow[c]{9}{*}{Bernoulli GLM} & \NREC (ours) & 0.802 & 0.694 & 0.601 \\
 & \REJABC & 0.994 & 0.976 & 0.941 \\
 & \NLE & 0.740 & 0.605 & 0.545 \\
 & \NPE & 0.863 & 0.678 & 0.559 \\
 & \NRE (\NREB) & 0.899 & 0.812 & 0.751 \\
 & \SMCABC & 0.991 & 0.981 & 0.818 \\
 & \SNLE & 0.634 & 0.553 & 0.522 \\
 & \SNPE & 0.855 & 0.614 & 0.525 \\
 & \SNRE (\SNREB) & 0.718 & 0.584 & 0.529 \\
\cline{1-5}
\multirow[c]{9}{*}{Bernoulli GLM Raw} & \NREC (ours) & 0.928 & 0.775 & 0.609 \\
 & \REJABC & 0.995 & 0.984 & 0.966 \\
 & \NLE & 0.870 & 0.939 & 0.951 \\
 & \NPE & 0.900 & 0.765 & 0.607 \\
 & \NRE (\NREB) & 0.915 & 0.834 & 0.777 \\
 & \SMCABC & 0.990 & 0.959 & 0.943 \\
 & \SNLE & 0.990 & 0.973 & 0.987 \\
 & \SNPE & 0.906 & 0.658 & 0.607 \\
 